## Model 4: K Nearest Neighbors

In [1]:
import pandas as pd
import numpy as np
import time

import matplotlib as mpl
import matplotlib.pyplot as plt

# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

save_figures = False
figure_path = "../figures/models/"
# df_init = pd.read_parquet("../data/init.parquet")
# df_before = pd.read_parquet("../data/before_split.parquet")
# df_train = pd.read_parquet("../data/prep_train.parquet")
# df_val = pd.read_parquet("../data/prep_val.parquet")
# df_test = pd.read_parquet("../data/prep_test.parquet")
# df_tt_train = pd.read_parquet("../data/prep_tt_train.parquet")
# df_tt_test = pd.read_parquet("../data/prep_tt_test.parquet")
df_all = pd.read_parquet("../data/prep_all.parquet")

In [2]:
df_all.head()

,Year_Y2015,Year_Y2016,Country_England,Country_Wales,Supermarket_Asda,Supermarket_Tesco Extra,Supermarket_Tesco Metro,Supermarket_Waitrose,Time_Evening,Time_Morning,...,YearCountryAge_Y2016EnglandAge_g2,YearCountryAge_Y2016EnglandAge_g3,YearCountryAge_Y2016WalesAge_g1,YearCountryAge_Y2016WalesAge_g2,YearCountryAge_Y2016WalesAge_g3,YearCountryAge_nan,ObsSize,FemaleN,MaleN,y
0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.471136,-1.156159,0.871226,1
1,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.471136,-1.156159,0.871226,1
2,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-0.471136,0.583858,-0.993069,1
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.811174,2.323876,-0.993069,1
4,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,-0.471136,0.583858,-0.993069,1


In [3]:
X_all = df_all.iloc[:,0:-1].to_numpy()
y_all = df_all["y"].to_numpy()

### KNN, Grid Search

In [4]:
random_state = 132
KNN_grid = KNeighborsClassifier(n_jobs = 1)
KNN_grid

KNeighborsClassifier(n_jobs=1)

In [5]:
params_KNN = {'n_neighbors': [2, 3, 5, 8, 9, 10, 11, 12, 15, 30], 
              'weights': ['uniform', 'distance']}

random_states_split = [123, 456, 789, 
                       741, 852, 963, 
                       159, 753, 951, 357]

gridCV_knn = GridSearchCV(estimator = KNN_grid, 
                          param_grid = params_KNN, 
                          scoring = 'balanced_accuracy', 
                          n_jobs = -2,
                          refit = True, 
                          cv = 5, verbose = 4)
gridCV_knn

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_jobs=1), n_jobs=-2,
             param_grid={'n_neighbors': [2, 3, 5, 8, 9, 10, 11, 12, 15, 30],
                         'weights': ['uniform', 'distance']},
             scoring='balanced_accuracy', verbose=4)

In [6]:
%%time

BalAccuracy_knn = []

for each_rs in random_states_split:
    start_time = time.time()
    X_other_temp, X_test_temp, y_other_temp, y_test_temp = \
        train_test_split(X_all, y_all, 
                         test_size = 0.1, 
                         random_state = each_rs, 
                         stratify = y_all)
    
    gridCV_knn.fit(X_other_temp, y_other_temp)
    
    BalAccuracy_knn.append(gridCV_knn.best_score_)
    
    print(gridCV_knn.best_params_, flush = True)
    
    end_time = time.time()
    print(f"Iteration {each_rs}: {end_time - start_time:.3f} seconds", flush = True)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 12, 'weights': 'uniform'}
Iteration 123: 2.659 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 8, 'weights': 'uniform'}
Iteration 456: 1.344 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 8, 'weights': 'distance'}
Iteration 789: 1.356 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 2, 'weights': 'uniform'}
Iteration 741: 1.348 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 8, 'weights': 'uniform'}
Iteration 852: 1.345 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 2, 'weights': 'uniform'}
Iteration 963: 1.315 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbors': 2, 'weights': 'uniform'}
Iteration 159: 1.386 seconds
Fitting 5 folds for each of 20 candidates, totalling 100 fits
{'n_neighbor

In [7]:
gridCV_knn.best_params_

{'n_neighbors': 2, 'weights': 'uniform'}

In [8]:
print(f"{np.mean(BalAccuracy_knn):.4f} ({np.std(BalAccuracy_knn):.4f})")

0.6151 (0.0101)


In [9]:
import json

knn_scores = {"knn": BalAccuracy_knn}

with open("../results/knn_scores.json", "w") as outfile:
    json.dump(knn_scores, outfile)
    
# with open("logistic.json", "r") as readfile:
#     dict_data = json.load(readfile)